# Background
This practical continues where the previous one left off. You will continue to investigate baseband communications between a transmitter and a receiver. However, in this practical, you will study the effect of noise on the reliability of data transmission. You will experimentally measure the increase in bit error probability as the noise in the channel increases, and compare it to the theoretically predicted value.

We will first create a simple transmitter and receiver to transmit the message "Hello, world!" encoded as a polar baseband signalling scheme. We will connect two computers using stereo audio cable (which has a left and right channel). The simple transmitter will initially transmit the line coded signal on the left channel, while a clock signal is transmitted on the right channel. The receiver will use the clock signal to decode polar line coded signal and retrieve the original message. We will then modify the receiver to extract the clock signal by self-synchronising on the line coded signal (i.e. just using the left-channel information).

After successfully transmitting and receiving the short message, we will modify the transmitter and receiver to transmit 100,000 bits of information. We will again initially use the clock signal sent on the right channel to decode the received signal. After successful decoding of the 100,000 bits, we will again modify the receiver to extract the clock signal by self-synchronisation.

Lastly, we will add noise so we can measure the bit error probability curve for polar signalling as a function of the signal-to-noise ratio.

# General instructions
- You will need *soldering* equipment for this practical.
- You will need two computers for this practical: one to use as a transmitter, one to use as a receiver.
- You're welcome to work on this practical on your own, or with a colleague.
- The practical consists of signal manipulation and noise generation in Python/MATLAB, the use of the sound card as a digital-to-analogue and analogue-to-digital converter, simple circuits to emulate a transmission channel, and the oscilloscope for signal analysis in the analogue domain.
- Each student should hand in his/her own report before 10:00 on Tuesday, 15 May 2018.
- Reports should be submitted on SUNLearn using the following link <a href="https://learn.sun.ac.za/mod/assign/view.php?id=596630">Practical 3 Report submission</a>
- Reports must be submitted in Word (.doc) or portable document format (.pdf).
- Identical reports are not acceptable — each student should hand in his/her own work.
- The lecturer and assistant will be available in the second-floor lab (E252) on Tuesday, 8 May from 14:00 to 16:00, to render assistance with the practical.
- A mark will be given for the practical report, which will contribute to your class mark for Telecommunications 414.
- Practicals are compulsory, and all practicals must be completed to achieve a class mark.

Note that your report should follow the style of a good technical report, with an Introduction, Background, Experiments and Conclusions.

In [ ]:
# Preliminaries
import matplotlib
matplotlib.use('nbagg')

#%matplotlib inline
%matplotlib notebook

import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.io import loadmat
import scipy.signal as signal
import IPython.display
import numpy as np
import scipy.stats as stats

plt.rcParams["figure.figsize"] = (9,5)

# 1. Transmitting 'Hello World' together with a clock signal

The transmitter implemented in the following code uses a 8-bit ASCII encoding to encode the message "Hello, world!". The line coded signal is sent on the left channel. The timing reference (clock signal) is sent on the right channel. 

## Generating the line coded signal
The line coded signal is constructed as follows: 
1. The message is preceded by a relatively long sequence of alternating zeros and ones (usually at least 256) to allow the receiver to synchronise. This is called a training sequence. We will use a training seqeunce of 32 bits.
    ~~~python
	D_training = np.kron(np.ones(16), np.array([1,0]));
    ~~~
	
2. The training sequence is followed by exactly eight binary ones to indicate the start of the frame. This is called the start-of-frame (SOF) character. 
	~~~python
	D_SOF = np.ones(8)
	~~~
3. The SOF is followed by the bit representation of the message. The bit representation of the message is also referred to as the payload. The message is converted into a bit representation (the payload) using the code below:
	~~~python
	# Encoding the message
	message = 'Hello, world!';
	 
	# Convert the ASCII values to unsigned integer values
	ascii = np.array([ord(c) for c in message], dtype=np.uint8)

	# Convert the ASCII values to bits
	D_payload = np.unpackbits(ascii)
	~~~
4. The end of the message is indicated by the end-of-frame (EOF) character (hexadecimal 0x1A). 
	~~~python
	D_EOF = np.array([0,0,0,1,1,0,1,0]) # This is the ASCII character 26 or hexadecimal 0x1A
	~~~
5. The complete bit sequence of the line coded message is constructed as follows:
	~~~python
	D = np.concatenate([D_training, D_SOF, D_payload, D_EOF]);
	~~~
6. As in the previous practical we use polar encoding for the line coded signal. The encoded signal is created in a similar manner as in the previous practical:
	~~~python
	Fs = 11025;       # Sampling frequency
	Rb = 1102.5;      # Data rate
	A = 1;            # Signal amplitude

	# Define the half-width, rectangular pulse shape
	p = A*np.array([1,1,1,1,1,0,0,0,0,0])

	# Define a pulse which is all zero. We will use this to add silence at the start and end of the line coded signal. We will use 128 bits of silence at the start and end of the line coded signal
	p_empty = np.zeros(10);
	D_empty = np.zeros(128);

	# Generate the line-coded signal
	a = 2*D-1;
	X_L = np.concatenate([np.kron(D_empty, p_empty), np.kron(a, p), np.kron(D_empty, p_empty)])
	~~~

## Generating the clock signal
We must also create the clock signal for the right channel. This is done in a similar manner as in the previous practical, but care must be taken to ensure that the left and right channel is the same length.

~~~python
# Determine the total number of symbols in the left channel
totalSymbols = len(a)+2*len(D_empty)
	
# Define the rectangular pulse shape for the clock signal. Note the offset.
p_clock = np.array([0,0,1,1,1,1,1,0,0,0]);
a_clock = np.ones(totalSymbols);

# Generate the clock signal
C = np.kron(a_clock, p_clock);  # Amplitudes are between 0 and 1
X_R = C*2-1;					# Amplitudes converted to be between -1 and 1.
~~~
   
## Putting it all together   
Now that we have generated the data for both the left and right channel, we combine the two channels and create an audio player.
~~~python
# Combine left channel and right channel
X = [X_L, X_R];
IPython.lib.display.Audio(rate=Fs, data=X)
~~~

In [ ]:
# Define the message to be transmitted
message = 'Hello, world!';

# 2. Transmitting and recording the line coded signal
In the first section we created a stereo signal with a polar line coded signal on the left channel and a clock signal on the right channel. This can now be transmitted using the first computer's sound card while the second computer can record and decode the transmitted message. The procedure is as follows:
1. Connect the first computer's speaker/headphone output to the second computer's microphone input by connecting a second stereo jack and cable to the breadboard. 
2. Recording audio in Python is not as simple as playing it. We will therefore use MATLAB to record the audio using an audio recorder object. Once we have recorded the message, we will save it as a '.mat' file which will then be loaded into the Python Jupyter notebook to decode the received message. In MATLAB, you can record audio by using an audiorecorder object.
The following sequence of commands sets up an audiorecorder object at a sample rate of 11025 sps, 16-bit quantisation, and using both channels:
~~~MATLAB
r = audiorecorder(11025, 16, 2)
~~~
Audio recording can be started and stopped by executing:
~~~MATLAB
record(r)
stop(r)
~~~
You can also use a blocking recorder to record a fixed number of seconds using the following commands:
~~~MATLAB
disp('Press any key to start recording')
pause()
recordblocking(r, secondsToRecord);
~~~
Finally, the recorded values can be captured in a MATLAB vector by executing:
~~~MATLAB
Y = getaudiodata(r)
~~~
It's easiest to execute the record and stop commands in the MATLAB console as needed, rather than putting them inside a script.
3. Use the audiorecorder object to verify that you can capture the transmitter's data signal on the receiver. Plot the received line coded signal and clock signal, and confirm that the right bit sequence is received.
~~~MATLAB
plot(Y(:,1)) % this will plot the left channel
plot(Y(:,2)) % this will plot the right channel
~~~
4. Once you have verified that you can capture the transmitter's data correctly you can save the data to a '.mat' file by executing the following MATLAB command:
~~~MATLAB
save('Y.mat','Y')
~~~
5. In the Jupyter notebook you can load the recorded data using the following Python command:
~~~python
from scipy.io import loadmat
Y = loadmat('Y.mat')['Y']
~~~
6. You can also save data from the Jupyter notebook into the MATLAB format using the following Python command:
~~~python
from scipy.io import savemat
Y = np.stack([X_L,X_R], axis=-1)
mat_dict = {}
mat_dict['Y'] = Y;
savemat('Y.mat',mat_dict)
~~~

# 3. Decoding the received message using the provided clock signal
We now should have the recorded data in the Jupyter notebook. The variable 'Y' should be Nx2 matrix, where the first column is the recorded line coded signal (left channel) and the second column is the provided clock signal (right channel). We are now ready to process the recorded data to decode the received message. Decoding the received message consists of:
1. Extracting the payload
2. Converting the bits of the payload back into a string

## Extracting the payload
Since we don't know how much silence is recorded before the line coded signal is received, we use the instantaneous received energy to decide when to start decoding by comparing it to a threshold. When the threshold is exceeded, the receiver  will start decoding. The receiver should initially look for a long sequence of alternating zeros and ones (at least 32 in total). Then, as soon as it receives 8 binary ones in succession, it has received the start-of-frame sequence (the SOF) and it knows that the payload signal follows next (in our implementation we only look for the SOF). The receiver can then decode the payload byte by byte until the EOF character is received. Since the receiver does not know the length of the payload, it cannot only check for the EOF character after receiving 8 new bits. It needs to check for the EOF character each time a new bit has been received. The following code implements the decoder:
~~~python
# Define the EOF marker
D_EOF = np.array([0,0,0,1,1,0,1,0]);

# Specify the maximum number of symbols that we can decode. It should be 128+32+8+13*8+8+128=408
totalSymbols = 408 # len(a)+2*len(D_empty)

# Get the recorded data
Y = loadmat('Y.mat')['Y']

# L is the received signal on the left channel. We also normalise it.
L = Y[:,0];
L = L/np.max(L); 

# C is the clock signal on the right channel. We don't need to normalise it since we will just look for zero-crossings
C = Y[:,1];

# N is the length of the data;
N = len(L);

# number of received bits
bit_counter = 0;
received_bit_counter = 0;

# Buffer for the decoded ASCII characters
current_char_bits = [];

# Have we started receiving pulses
startDecoding = False;
energyThreshold = 0.5;

# Have we received the SOF character
SOF = False;
SOF_bit_counter = 0;

# Have we received the end-of-frame character
EOF = False;

# Buffer for storing the sampling instants so we can plot it
received_bits = np.zeros(int(np.floor(numBits*1.1)), np.int16);
sampling_instants = np.zeros(N);
offset = 2;
signalIndex = 0;
           
for i in range(2,np.min([(N-offset),N])):
    # We want to determine whether we have started receiving pulses before
    # starting to decode the received pulses. We start decoding when the instantaneous
    # energy exceeds some threshold
    instantaneousEnergy = L[i]*L[i];
    if (instantaneousEnergy > energyThreshold):
        if (startDecoding == False):
            signalIndex = i;
        startDecoding = True;

    # We want to find the positive gradient zero crossings to determine whether
    # it is decision instant
    ZC = ((C[i-1]*C[i] <= 0) and ((C[i-1] < 0) | (C[i] >= 0)));
    # If a zero-crossing then make a decision regarding bit
    if ((ZC == 1) and startDecoding):
        sampling_instants[i+offset] = 1;
        if (L[i+offset] > 0):
            bit = int(1);
        else:
            bit = int(0);            
        bit_counter = bit_counter + 1;
        if (SOF == False):
            # We need to receive 8 binary '1' in a row to start decoding. 
            if (bit == 1):
                SOF_bit_counter = SOF_bit_counter + 1;
                if (SOF_bit_counter == 8):
                    print('Frame start sequence detected.');
                    SOF = True;
            else: 
                SOF_bit_counter = 0;
                SOF = False;
                
        else:
            # Since we have received the SOF character, store the received payload bit
            received_bits[received_bit_counter] = bit;
            received_bit_counter = received_bit_counter+1;

            # Display a progress message for every 10000 bits decoded
            if (np.mod(received_bit_counter,10000) == 0):
                print('Received bits: %d' % received_bit_counter);
                            
            # Check that we have received at least 8 bits and check whether the
            # last 8 bits are the EOF character
            current_char_bits.append(bit);
            if (len(current_char_bits) == 8):
            	# Decode the last 8 bits as an ASCII character            
                c = np.packbits(current_char_bits, axis=-1);
                # Discard the least recently received bit from the EOF check buffer
                current_char_bits = current_char_bits[1:];
                # Check whether we have received the EOF char indicating the
                # decoded sequence is complete
                if (c == 26):
                    print('Frame end sequence detected.');
                    EOF = True;                
        
    if (EOF == True):
        break
                
payload = received_bits[:(received_bit_counter-8)]
~~~

In [ ]:
# Load the received signal that was recorded using MATLAB and decode the payload

## Displaying the polar and clock signals
When veryfying that the system is working correctly it is useful to display the received signals. Specifically we want to verify that the clock signal is correct and that the sampling instants are in the middle of the received pulses. The sampling instants are marked by the decoder and the following code can be used to display the received signals:
~~~python
indexes, = np.nonzero(sampling_instants)
left_handle, = plt.plot(L, label='Line coded signal')
clock_handle, = plt.plot(C, label='Clock signal')
sampling_handle = plt.stem(indexes,L[indexes],'r', label='Sampling instants');
plt.legend(handles=[left_handle, clock_handle, sampling_handle])
~~~

In [ ]:
# Plot the polar signal, clock signal and sampling instants

## Converting the payload into a string
To convert the bits of the payload back into a string we have to reverse the process of encoding the payload. We first convert each 8 bits into it's ASCII value (an unsigned integer). We then convert the ASCII characters into characters and finally concatenate all the characters into a single string. The following code performs this conversion:
The bit sequence representing the message (the payload) is converted back into a string using the code below:
~~~python
# Convert the bits to ASCII values
decoded_ascii = np.packbits(payload)

# Convert the ASCII values to a list of characters
characterlist = [chr(val) for val in decoded_ascii];

# Convert the list of characters into a single string
decoded_message = ''.join(characterlist)
print('Received message %s' % decoded_message)
~~~

In [ ]:
# Convert the payload back into a single message string

# 4. Decoding the received message using self-sinchronisation
Instead of using the provided clock signal on the right channel, we can convert the receiver so that it no longer requires the reference clock signal, but instead extracts the clock signal directly from the received polar signal. Extracting the clock signal entails the follows:
1. Rectify the received polar signal rectifying by taking the absolute value.
    ~~~python
    R = np.abs(L);
    ~~~
2. Filter the rectified polar signal very narrowly on the expected clock frequency by using the BUTTER and FILTER functions of SciPy to filter the clock to obtain a clean sine wave. The positive-gradient zero crossings of the clean sine wave indicate time instant when the received signals should be sampled (take care to correct for phase shift). 
    ~~~python
    Fs = 11025;       # Sampling frequency
    Rb = 1102.5;      # Data rate
    wn = Rb*2./Fs;    # normalized frequence

    # 4th order butterword filter centered around clock signal
    filterOrder = 4;
    B,A = signal.butter(filterOrder, np.array([0.95*wn, 1.05*wn]),'bandpass')

    # C is the extracted clock signal created by filtering the received clock with a zero-phase digital filter
    # Since the zero-phase filter effectively filters the signal twice (forwards and backwards) - we use a 
    # 4th order Butterworth filter resulting in an effective 8th-order zero-phase filter;
    C = signal.filtfilt(B,A,R);
~~~
3. Use the extracted clock signal to decode the received bits and extract the payload. This can be done by simply replacing the clock signal in the receiver code.

In [ ]:
# Modify the previous decoder to use self-sinchronisation instead of the provided clock signal

In [ ]:
# Plot the polar signal, clock signal and sampling instants

In [ ]:
# Convert the payload back into a single message string

# 5. Transmitting and receiving a packet of 100,000 bits

We know that the receiver needs some time to synchronise to the symbol clock of the received signal. Modify your transmitter so that you can transmit a packet of 100,000 bits at a time. This is done by replacing the payload with randomly generated bits, as explained below. 

## Random data generation
Python provides a handy way to generate pseudorandom bits in a way that can be replicated at both the transmitter and the receiver, so that the receiver can easily check whether the correct pseudorandom (PN) sequence was received. 
- The following Python code generates a pseudorandom sequence of a hundred bits:
    ~~~python
    np.random.seed(1234);
    data = np.where(np.random.normal(0.0, 1.0, 100) > 0, 1, 0);
    ~~~
    The first line initialises the pseudorandom number generator. The second line configures the number of bits that will be generated and generates the bits. 
- The following Python command resets the pseudorandom number generator and generates a sequence identical to the first one.
    ~~~python
    np.random.seed(1234);
    data = np.where(np.random.normal(0.0, 1.0, 100) > 0, 1, 0);
    ~~~
Two PN generators on different machines will also generate the same sequence.

## Verifying correct transmission of the packet by counting bit errors
We now want to verify that we can transmit and receive the packet containing 100,000 bits of payload data. 
You should now do the following:
1. Generate a packet that is a new line coded signal that contains a payload of 100,000 bits
2. Transmit the packet from one computer and record it on the second computer using MATLAB. Save the recorded data to a '.mat' file.
3. Load the recorded data into the Jupyter notebook and decode the payload as you did for the 'Hello, World!' message.
4. Compare the transmitted payload to the received payload. You should get zero bit errors. If you have two bit sequences A and B, and you want to count the number of places in which they differ (e.g. the number of bit errors). In Python the following will the number of places where the two sequences differ
    ~~~python
    errors = np.sum(np.abs(A-B))
    BER = errors/numPayloadBits
    ~~~



In [ ]:
# Generate and transmit a packet containing 100,000 bits
# the left channel and a clock signal on the right channel

In [ ]:
# Load the received signal that was recorded using MATLAB and decode the payload

In [ ]:
# Display a short interval of the received signals and verify that the 
# sampling instants are in the middle of the received pulses

In [ ]:
# Determine the number of bit errors that occurred during transmission

# 6. Adding noise to measure bit-error-rate curves
Now that a reference clock signal is no longer needed, the right channel can be used to artificially generate a noise signal. Generate white Gaussian noise with a standard deviation ten times lower than the signal amplitude. 

## Generating white Gaussian noise
A noise signal can be generated using the following Python commands:
~~~python
sigma = 0.1*A;
noise = np.random.normal(0.0, sigma, len(X_L));
~~~
Take care to generate noise that is the same length as the generated polar signal.

## Adding noise to communication channel
Add the signal and noise together in the analogue domain. You can use an op-amp, but that's overkill. A resistive adding circuit is much simpler. Adding the signals in the digital domain is the easy way out – only take that option if you get stuck, or cannot get the self-synchronisation working.


## Measuring bit error probability curves
Measuring bit error probability curves requires that packets are transmitted and received for various signal-to-noise ratios. For each signal-to-noise ratio, you must measure the average bit error rate by counting the number of bit error that occur during transmission. The procedure is as follows:

1. Separately measure the signal strength and noise strength at the receiver. Note that, in a practical receiver, the signal is usually so much stronger than the noise, that you can disregard the noise power when measuring the signal power. To measure the noise power, you switch off the signal (i.e. you can measure the noise power in the silence before and after the polar signal occurs in the recorded data). 

2. Transmit packets of 100,000 bits at different signal-to-noise ratios ($\sigma$ = 0.10A to 0.50A should be reasonable), and measure the average bit error probability. Plot it, and compare it to the theoretically predicted values for polar signalling. 

Take note the following:
1. For polar signalling the probability of error is given by
    $P_{b} = Q\left(\frac{A_p}{\sigma}\right)$ where $A_p$ is the amplitude of the received pulse (after normalisation) and $\sigma$ is the standard deviation of the noise (after normalisation). 
2. $Q(x)$ can be determined by using the following Python function:
    ~~~python
    stats.norm.sf(x)
    ~~~
3. The standard deviation of the noise can be measured by measure the noise power $N_i$ and noting that $N_i = \sigma^2$
4. For polar signalling the received amplitude can be estimated by one of two methods:
    1. Estimating the mean amplitude by measuring the amplitude at all the sampling instants.
    2. Calculating the mean amplitude from the received signal power. Note that for polar signalling and rectangular, half-width pulses the received signal power:
    
    $S_i = E_p R_b = A_p^2 T_b R_b = A_p^2$
5. If you do not want to disregard the noise when measuring the signal power, note that the noise and polar signal is statistically independent. The total received power at the receiver input is thus:
    
    $S_r = S_i + N_i$ 

# Wrapping up
Write the Conclusions section of your report, and submit on SUNLearn before 10:00 on Tuesday, 15 May 2018 using the following link <a href="https://learn.sun.ac.za/mod/assign/view.php?id=596630">Practical 3 Report submission</a>.

In [ ]:
# Generate the transmitted pseudorandom sequence of a 100,000 bits

# Compare the decoded payload with the transmitted payload and count the number of bit errors

# Determine the probability of bit error

In [ ]:
# Determine the received noise power

# Determine the received signal power

# Determine the signal to noise ratio                  

In [ ]:
# Determine the average received pulse amplitude Ap

# Determine the standard deviation of the noise

# Compare the measure bit error probability with the theoretically predicate value